In [1]:
import pandas as pd
import keras


## Change files to CSV

In [18]:
res = []

import os
dir_path = r'C:/Users/Harish Vasanth/Desktop/Machine-learning-projects/GroundUP/Datasets-20230714T035808Z-001/Datasets/Audios_split/' ## Add path to your folder
for path, _, files in os.walk(dir_path):
    # check if current path is a file
    print(path)
    print(files)
    res.append(files)


off_files =[]
on_files = []
for file in res[0] :
        if "(Off)" in str(file):
              off_files.append(file)
        else :
              on_files.append(file)
    



C:/Users/Harish Vasanth/Desktop/Machine-learning-projects/GroundUP/Datasets-20230714T035808Z-001/Datasets/Audios_split/
['Machine13(Off)_1.mp3', 'Machine13(Off)_10.mp3', 'Machine13(Off)_11.mp3', 'Machine13(Off)_12.mp3', 'Machine13(Off)_13.mp3', 'Machine13(Off)_14.mp3', 'Machine13(Off)_15.mp3', 'Machine13(Off)_16.mp3', 'Machine13(Off)_17.mp3', 'Machine13(Off)_18.mp3', 'Machine13(Off)_19.mp3', 'Machine13(Off)_2.mp3', 'Machine13(Off)_20.mp3', 'Machine13(Off)_21.mp3', 'Machine13(Off)_22.mp3', 'Machine13(Off)_23.mp3', 'Machine13(Off)_24.mp3', 'Machine13(Off)_25.mp3', 'Machine13(Off)_26.mp3', 'Machine13(Off)_27.mp3', 'Machine13(Off)_28.mp3', 'Machine13(Off)_29.mp3', 'Machine13(Off)_3.mp3', 'Machine13(Off)_30.mp3', 'Machine13(Off)_31.mp3', 'Machine13(Off)_32.mp3', 'Machine13(Off)_33.mp3', 'Machine13(Off)_34.mp3', 'Machine13(Off)_35.mp3', 'Machine13(Off)_36.mp3', 'Machine13(Off)_37.mp3', 'Machine13(Off)_38.mp3', 'Machine13(Off)_39.mp3', 'Machine13(Off)_4.mp3', 'Machine13(Off)_40.mp3', 'Machine

In [20]:
import csv
with open('data.csv', 'w', newline='') as file: ## add your csv file
     writer = csv.writer(file)
     writer.writerow(["Audio","State","NumericState"])

     for row in off_files :
               writer.writerow([row, "off",0])
     for row in on_files :
               writer.writerow([row, "on",1])

df = pd.read_csv('data.csv')



TypeError: open() got an unexpected keyword argument 'newline'

In [4]:
df

,Audio,State,NumericState
0,Machine13(Off)_1.mp3,off,0
1,Machine13(Off)_10.mp3,off,0
2,Machine13(Off)_11.mp3,off,0
3,Machine13(Off)_12.mp3,off,0
4,Machine13(Off)_13.mp3,off,0
...,...,...,...
115,Machine13(On)_6.mp3,on,1
116,Machine13(On)_60.mp3,on,1
117,Machine13(On)_7.mp3,on,1
118,Machine13(On)_8.mp3,on,1


### Feature extraction

In [6]:
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import librosa

In [7]:
def classify_on_off(audiofile : str, machine: str):
    assert machine[0] == "M"
    y, sr = librosa.load(audiofile, sr=None)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=8).T,axis=0)
    mfcc_0 = list(mfcc)[0]
    if machine in mfcc_thresh_dict:
        threshold = mfcc_thresh_dict[machine]
    else:
        threshold = -515

    if mfcc_0 > threshold:
        status = "ON"
    else:
        status = "OFF"

    return status

In [8]:
data = pd.read_csv('data.csv') # Alter accordingly
X = data['Audio']
y = data[['State','NumericState']]
audio_dataset_path= dir_path
#Initialization

In [9]:
def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)

def features_extractor(file):
    audio, sample_rate = librosa.load(file_name) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features



In [11]:

### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]

for index_num,row in tqdm(data.iterrows()):
    file_name = os.path.join(audio_dataset_path + str(row["Audio"]))
    final_class_labels=row["NumericState"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

AttributeError: 'numpy.ndarray' object has no attribute 'iterrows'

### Test Train  Validation split

In [12]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1)

len(X_train) +len(X_val) +len(X_test) 

120

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_val,y_val)    
clf.score(X_test,y_test)    

y_predict = clf.predict(X_test)

In [17]:
from sklearn.metrics import classification_report
classification_report(y_test,y_predict)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00         8\n           1       1.00      1.00      1.00        16\n\n    accuracy                           1.00        24\n   macro avg       1.00      1.00      1.00        24\nweighted avg       1.00      1.00      1.00        24\n'